<a href="https://colab.research.google.com/github/lin-tan/eagle/blob/main/tensorflow/rule_14/rule_14_tf_bug_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tensorflow==2.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached tensorflow-2.2.0-cp37-cp37m-manylinux2010_x86_64.whl (516.2 MB)
  Using cached tensorboard-2.2.2-py3-none-any.whl (3.0 MB)
  Using cached tensorflow_estimator-2.2.0-py2.py3-none-any.whl (454 kB)
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estimator-2.7.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.7.0+zzzcolab20220506150900
    Uninstalling tensorflow-2.7.0+zzzcolab20220506150900:
      Successfully uninstalled tensorflow-2.7.0+zzzcolab20220506150900


In [ ]:
import tensorflow as tf
import numpy as np
print(tf.__version__)

# The following code adds padding to the input and then deletes the padding. 
# Therefore the input and the output should be equal.

argument = {'padding': 35}
input = np.arange(9, dtype=np.float32).reshape(1, 3, 3, 1)

pad_fun = tf.keras.layers.ZeroPadding2D
unpad_fun = tf.image.extract_glimpse

# pad input
layer_1 = pad_fun(**argument)
padded_input = layer_1(input)
# print(padded_input.shape)
padding = argument["padding"]
if isinstance(argument["padding"], int):
    padding = ((argument["padding"], argument["padding"]), (argument["padding"], argument["padding"]))
elif len(argument["padding"]) == 2:
    if isinstance(argument["padding"][0], int):
        padding = ((argument["padding"][0], argument["padding"][0]), (argument["padding"][1],
                                                                      argument["padding"][1]))
    elif len(argument["padding"][0]) == 2:
        padding = argument["padding"]

# unpad input and compare
argument_2 = {}  # convert argument accordingly
argument_2["centered"] = False
argument_2["normalized"] = False
argument_2["input"] = padded_input
argument_2["size"] = input.shape[1:3]

offset_i = [padding[0][0], padding[1][0]]
offsets = []
for i in range(input.shape[0]):
    offsets.append(offset_i)
argument_2["offsets"] = offsets
# print(argument_2)

unpadded_input = unpad_fun(**argument_2)

output_1 = input
output_2 = unpadded_input
print(output_1)
print(output_2)  # The two outputs are inconsistent, which indicates a bug
# This inconsistency disappears in tf 2.7.0

print("if output_1 equals output_2: ", np.allclose(output_1, output_2, rtol=1e-05, atol=1e-02))

2.2.0
[[[[0.]
   [1.]
   [2.]]

  [[3.]
   [4.]
   [5.]]

  [[6.]
   [7.]
   [8.]]]]
tf.Tensor(
[[[[0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]]

  [[0.]
   [0.]
   [0.]]]], shape=(1, 3, 3, 1), dtype=float32)
